In [3]:
import pandas as pd

import pandas as pd
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv("partidos_colombia_2024.csv")

In [4]:
print("\n🔹 Valores únicos en la columna 'Marcador':")
print(df["Marcador"].unique())



🔹 Valores únicos en la columna 'Marcador':
['Alianza Petrolera' 'Deportivo Pereira' 'Deportes Tolima'
 'Atlético Bucaramanga' 'Once Caldas' 'América de Cali'
 'Atlético Nacional' 'Boyacá Chicó' 'Deportivo Cali' 'Rionegro Águilas'
 'La Equidad' 'Santa Fe' 'Fortaleza CEIF' 'Independiente Medellín'
 'Deportivo Pasto' 'Millonarios' 'Junior FC' 'Llaneros' 'Unión Magdalena'
 'Envigado']


In [5]:
print(df.head())  # Mostrar las primeras filas
print("\n🔹 Columnas del DataFrame:")
print(df.columns)  # Mostrar los nombres de las columnas



        Fecha  Equipo Local           Marcador Equipo Visitante
0  26-01-2025           NaN  Alianza Petrolera            1 - 1
1  22-02-2025           NaN  Alianza Petrolera            3 - 2
2  07-03-2025           NaN  Alianza Petrolera            1 - 0
3  02-02-2025           NaN  Deportivo Pereira            0 - 0
4  07-02-2025           NaN    Deportes Tolima            3 - 1

🔹 Columnas del DataFrame:
Index(['Fecha', 'Equipo Local', 'Marcador', 'Equipo Visitante'], dtype='object')
